In [1]:
!pip install pywikihow
!pip install transformers
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-bqh5ad6p
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-bqh5ad6p


In [2]:
import pandas as pd
from pywikihow import RandomHowTo, HowTo
from transformers import pipeline

#Importar Parrot (paraphraser)

from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

#Resume
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#WikiHow
how_to = HowTo("https://www.wikihow.com/Study")

In [3]:
#Generar una semilla para la recreación de las frases
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

#Cargar el modelo parrot
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [4]:
#Read data
#post = pd.read_excel("WikiHow.xlsx", index_col=0)

In [5]:
#Create Dataset
#ERASE THE PREVIOUS DATASET
data = {"Title" : ["Titulo prueba"], "Summary": ["Summary prueba"]}
post = pd.DataFrame(data)
post.to_excel("SummaryParaphraser.xlsx")

In [6]:
#Get resumes and paraphrases
for step in how_to.steps:
  dat = step.as_dict()
  resume = summarizer(dat["description"], max_length=60, min_length=5, do_sample=False)
  para_phrases = parrot.augment(input_phrase=resume[0]["summary_text"], max_return_phrases = 2, do_diverse=True)
  para1 = ""
  para2 = ""
  n = 0
  for para_phrase in para_phrases:
    if (n == 0) and (not(para_phrase == ""))and (para_phrase[1] > 60):
      prueba = {"Title":dat["summary"], "Summary": para_phrase[0]}
      post = post.append(prueba, ignore_index=True)
      n += 1
    elif (not(para_phrase == "")and (para_phrase[1] > 60)):
      prueba = {"Title":dat["summary"], "Summary": para_phrase[0]}
      post = post.append(prueba, ignore_index=True)
  prueba = {"Title":dat["summary"], "Summary": resume[0]["summary_text"]}
  post = post.append(prueba, ignore_index=True)
  # print(dat)

Your max_length is set to 60, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 60, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


TypeError: ignored

In [ ]:
#SAVE
post.to_excel("SummaryParaphraser.xlsx")